In [6]:

import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.utils import to_scipy_sparse_matrix
import numpy as np
import scipy.sparse as sp
from scipy.sparse import csgraph
from scipy.sparse.linalg import eigsh  # For large sparse matrices
from numpy.linalg import eigh  # For dense matrices
from torch import nn


# Task 1: Implement Laplacian Positional Encodings (LapPE)
 - Compute Laplacian eigenvectors using the graph Laplacian matrix `L = D - A`.
 - Use `eigh` for efficient eigendecomposition (optimized for Hermitian matrices).
 - Handle sign ambiguity in eigenvectors (random flipping or SignNet).
 - Ensure embeddings are robust to sign flips and can capture graph structure.

In [7]:

def compute_lap_pe(data, pe_dim=10, use_signnet=False):
    """
    Compute Laplacian Positional Encodings (LapPE) for a given graph data.

    Args:
        data (torch_geometric.data.Data): Graph data object.
        pe_dim (int): Number of eigenvectors to compute.
        use_signnet (bool): Whether to handle sign ambiguity using SignNet.

    Returns:
        data (torch_geometric.data.Data): Graph data object with 'lap_pe' attribute added.
    """
    num_nodes = data.num_nodes
    edge_index = data.edge_index

    # Convert to scipy sparse matrix
    adj = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes).astype(float)

    # Compute the Laplacian matrix
    laplacian = csgraph.laplacian(adj, normed=False)

    # For large sparse matrices, use eigsh
    try:
        # Compute the smallest k+1 eigenvalues and eigenvectors
        eigvals, eigvecs = eigsh(laplacian, k=pe_dim+1, which='SM')
        eigvecs = eigvecs[:, eigvals.argsort()]  # Sort eigenvectors
    except RuntimeError:
        # Fall back to dense computation for small graphs
        laplacian_dense = laplacian.toarray()
        eigvals, eigvecs = eigh(laplacian_dense)
        eigvecs = eigvecs[:, eigvals.argsort()]

    # Exclude the first eigenvector (corresponding to the smallest eigenvalue)
    pe = eigvecs[:, 1:pe_dim+1]

    # Handle sign ambiguity
    if use_signnet:
        # SignNet implementation (placeholder)
        # You would implement SignNet as a neural network layer in your model
        # For now, we'll assume the sign ambiguity is handled during model training
        pass
    else:
        # Randomly flip signs during training (data augmentation)
        # This helps the model to be robust to sign flips
        sign_flip = np.random.choice([-1, 1], size=(pe.shape[1],))
        pe = pe * sign_flip

    # Convert to torch tensor
    pe_tensor = torch.from_numpy(pe).float()

    # Add to data
    data.lap_pe = pe_tensor

    return data





# Task 2: Combine LapPE with SignNet
   - Implement SignNet to make the model sign-invariant.
   - Use SignNet to process the eigenvectors of the Laplacian to produce consistent embeddings.


In [8]:
class SignNet(nn.Module):
    """
    SignNet: A neural network that is invariant to the sign flips of input eigenvectors.
    """
    def __init__(self, in_dim, hidden_dim, out_dim, num_layers=2):
        super(SignNet, self).__init__()
        self.num_layers = num_layers
        self.mlp = nn.ModuleList()
        for _ in range(num_layers):
            self.mlp.append(nn.Linear(in_dim if _ == 0 else hidden_dim, hidden_dim))
        self.out_layer = nn.Linear(hidden_dim, out_dim)

    def forward(self, x):
        """
        Forward pass for SignNet.

        Args:
            x (torch.Tensor): Input eigenvectors of shape (num_nodes, pe_dim)

        Returns:
            torch.Tensor: Sign-invariant embeddings of shape (num_nodes, out_dim)
        """
        # Compute both x and -x
        x_pos = x
        x_neg = -x

        # Pass through MLP separately
        for layer in self.mlp:
            x_pos = F.relu(layer(x_pos))
            x_neg = F.relu(layer(x_neg))

        # Sum the outputs to get sign-invariant representation
        x = x_pos + x_neg

        # Final output layer
        x = self.out_layer(x)

        return x

def compute_lap_pe_with_signnet(data, pe_dim=10, hidden_dim=64, out_dim=32, num_layers=2):
    """
    Compute Laplacian Positional Encodings and process them with SignNet.

    Args:
        data (torch_geometric.data.Data): Graph data object.
        pe_dim (int): Number of eigenvectors to compute.
        hidden_dim (int): Hidden dimension for SignNet.
        out_dim (int): Output dimension for SignNet embeddings.
        num_layers (int): Number of layers in SignNet.

    Returns:
        data (torch_geometric.data.Data): Graph data object with 'sign_inv_lap_pe' attribute added.
    """
    num_nodes = data.num_nodes
    edge_index = data.edge_index

    # Convert to scipy sparse matrix
    from torch_geometric.utils import to_scipy_sparse_matrix
    import numpy as np
    from scipy.sparse import csgraph
    from scipy.sparse.linalg import eigsh
    from numpy.linalg import eigh

    adj = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes).astype(float)

    # Compute the Laplacian matrix
    laplacian = csgraph.laplacian(adj, normed=False)

    # For large sparse matrices, use eigsh
    try:
        eigvals, eigvecs = eigsh(laplacian, k=pe_dim+1, which='SM')
        eigvecs = eigvecs[:, eigvals.argsort()]  # Sort eigenvectors
    except RuntimeError:
        laplacian_dense = laplacian.toarray()
        eigvals, eigvecs = eigh(laplacian_dense)
        eigvecs = eigvecs[:, eigvals.argsort()]

    # Exclude the first eigenvector (corresponding to the smallest eigenvalue)
    pe = eigvecs[:, 1:pe_dim+1]

    # Convert to torch tensor
    pe_tensor = torch.from_numpy(pe).float()  # Shape: (num_nodes, pe_dim)

    # Initialize SignNet
    signnet = SignNet(in_dim=pe_dim, hidden_dim=hidden_dim, out_dim=out_dim, num_layers=num_layers)

    # Process the positional encodings with SignNet
    sign_inv_pe = signnet(pe_tensor)  # Shape: (num_nodes, out_dim)

    # Add to data
    data.sign_inv_lap_pe = sign_inv_pe

    return data
